In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit


In [5]:
def lin_func(x,a):

    return x*a

power_uW = np.array([217.2,211.2,194.4,198.1,219.1,219.3,216.3,208.6,205.6])
voltage_mV = np.array([321.31,308.86,285.55,295.06,321.86,322.09,312.31,302.36,299.30])

popt,pcov = curve_fit(lin_func,power_uW,voltage_mV)
conversion_factor = popt[0]
conversion_offset = 0

# plt.plot(power_uW,voltage_mV,'.')
# plt.plot(power_uW,lin_func(power_uW,popt[0]))
# plt.xlabel('Optical power [µW]')
# plt.ylabel('Voltage [mV]')

In [6]:
def get_paths_homodyne(directory):
    
    filenames = os.listdir(directory)
    return [directory + "\\" + e for e in filenames if "homodyne" in e]

def get_paths_intensity(directory):
    
    filenames = os.listdir(directory)
    return [directory + "\\" + e for e in filenames if e.endswith("esa.txt")]#[directory + "\\" + e for e in filenames if e.endswith("esa.txt")][:-10]

def get_OSA_paths(directory):
    
    filenames = os.listdir(directory)
    return [directory + "\\" + e for e in filenames if "OSA" in e][:-10]


def get_data(path,length=1):

    return np.loadtxt(path, encoding = "unicode_escape",skiprows=1,delimiter=' ')

def plot_data(path,figure_no=1):

    header = get_header(path)[0]
    xs, ys = get_data(path)
    plt.figure(figure_no)
    plt.plot(xs*1e-9,ys)
    if len(header)>3:
        plt.title(f'{header[0]}, {header[1]}, {header[2]}, {header[3]}')
    else:
        plt.title(f'{header[0]}, {header[1]}, {header[2]}')
    plt.xlabel('freq, f [GHz]')
    

def ratio_to_db(feedback_power,output_power,):

    if (feedback_power == 0.0) or (feedback_power == 0):
        feedback_power = 0.000003 #0.003 nW (minimum for feedback PM)
    if (output_power == 0.0) or (output_power == 0):
        output_power = 0.000003

    return 10*np.log10(feedback_power/output_power)

def linear_to_dB(datapoint_linear):

    return 10*np.log10(datapoint_linear)


def dB_to_linear(power):
    pow_lin = 10**(power/10)
    return pow_lin

def OSA_fb_dB(feedback_power,peak_power,):

    peak_power_uW = 10**(peak_power/10)*1000 * 40 #in µW, multiplied by 40 to get correct value

    if feedback_power == 0.0:
        feedback_power = 0.000003 #0.003 nW (minimum for feedback PM)

    fb_dB = ratio_to_db(feedback_power,peak_power_uW)

    return fb_dB


def convert_optical_to_electrical(power, ret_V = False, conversion=371.7*1e-3/208): #Use power directly from PM (i.e. not multiplied by 40), and return average optical power squared in dBm


    # conversion = 371.7*1e-3/208 #V/µW      #307.4*1e-3/104.9 #V/µW      Conversion rate of PM power in µW to volts

    volts = conversion*power #V     Convert optical PM power to volts in ESA

    if ret_V:
        return volts
    
    else:

        elec_power = volts**2/50 #V^2/Ohm = W/Hz        Convert to electrical power

        elec_power_mW = 1e3*elec_power #mW/Hz           Make it mW

        power_sq_dBm = 10*np.log10(elec_power_mW/1) # 10*log10( [P/1mW]^2 ) [dBm]       Take the average power, square it and make it into dBm

        return power_sq_dBm
        


def convert_optical_to_electrical2(power, ret_V = False, conversion=1.9/1950): #Use power directly from PM (i.e. not multiplied by 40), and return average optical power squared in dBm

    #power = 1020 #Same for all as it is the EDFA who determines this

    #conversion = 1.9/1950 #V/µW

    volts = conversion*power #V     Convert optical PM power to volts in ESA

    if ret_V:
        return volts
    
    else:

        elec_power = volts**2/50 #V^2/Ohm = W/Hz        Convert to electrical power

        elec_power_mW = 1e3*elec_power #mW/Hz           Make it mW

        power_sq_dBm = 10*np.log10(elec_power_mW/1) # 10*log10( [P/1mW]^2 ) [dBm]       Take the average power, square it and make it into dBm

        return power_sq_dBm




def get_header(path,length=1):
    lines = []
    with open(path,encoding="ISO-8859-1") as file:
        for i in range(length):
            line = file.readline()
            lines.append(line[1:].split(','))
    return lines


def esa_header_data(header):

    length = len(header[0])
    
    output_power = float(header[0][0].split(" ")[1][:-2])*40 

    fb_level = float(header[0][1].split(" ")[3][:-2])
    
    if length == 6:
        gain = header[0][2][1:]

        pol = header[0][3].split(" ")[2]

        if 'MHz' in header[0][4].split(" ")[1]:
            rbw = float(header[0][4].split(" ")[1][4:-3])*1e6
        elif 'kHz' in header[0][4].split(" ")[1]:
            rbw = float(header[0][4].split(" ")[1][4:-3])*1e3
        else:
            rbw = float(header[0][4].split(" ")[1][4:-2])
        
    elif length == 5:
        pol = header[0][2].split(" ")[2]

        if 'MHz' in header[0][3].split(" ")[1]:
            rbw = float(header[0][3].split(" ")[1][4:-3])*1e6
        elif 'kHz' in header[0][3].split(" ")[1]:
            rbw = float(header[0][3].split(" ")[1][4:-3])*1e3
        else:
            rbw = float(header[0][3].split(" ")[1][4:-2])

        gain = 'None'

    if pol == 'Misaligned':
        pol = 'misaligned'

    if pol == 'Aligned':
        pol = 'aligned'

    if pol == 'None':
        pol='none'


    return output_power, fb_level, gain, pol, rbw


def scientific(x, pos):
    if x == 1e5:
        return r'$10^5$'
    elif x == 1e6:
        return r'$10^6$'
    elif x == 1e7:
        return r'$10^7$'
    else:
        return r'$%d \times 10^{%d}$' % (int(x / (10**int(np.log10(x)))), int(np.log10(x)))

In [10]:
def plot_intensity_data_GHz(directory):

    paths = get_paths_intensity(directory)

    fb_counter = [0,0,0,0,0,0,0,0]

    no_fb_rel_osc = [0,0,0,0,0,0]
    
    colors = ['#377eb8','#4daf4a','#ff7f00', '#e41a1c',
                  '#f781bf', '#a65628', '#984ea3',
                  '#999999', '#dede00']

    for i, path in enumerate(paths):

        header = get_header(path,length=1)
        print(path)

        power, fb_level, _, pol, rbw = esa_header_data(header)

        rbw_db = 10*np.log10(rbw) #Hz

        fb_dB = ratio_to_db(fb_level,power)

        xs, ys = get_data(path) #Hz, dBm

        ys += -rbw_db #dBm/Hz   Remove raw data dependence on bandwidth 

        ys_lin = dB_to_linear(ys) #Make data linear

        ys = 10*np.log10(ys_lin) #dBm/Hz    Make data into log scale


        ys += -convert_optical_to_electrical(power/40,conversion=352*1e-3/99.2)  #Divide with the average optical power (in dB here, so subtract)


        plt.figure(0, figsize=(9.6,4.8),dpi=100)

        pol = pol[:2]
        
        for levels in [-9,-24,-93]:
            if abs(fb_dB-levels)<2:

                plt.plot(xs*1e-9,ys, color=colors[fb_counter[7]],alpha = 1-fb_counter[7]/10,label=f'fb: {fb_dB:.1f} dB'+ ',' + pol)

                fb_counter[7] +=1

        plt.grid()

                
    # save_path = r"some\path\here"

    plt.xlabel('Frequency [GHz]')
    plt.ylabel('RIN [dBc/Hz]')
    plt.ylim([-160,-91])
    plt.xlim([0,10])
    plt.xticks([0,1,2,3,4,5,6,7,8,9,10])

    plt.legend(['No FB','Aligned -9 dB','Aligned -22 dB', 'Misaligned -9 dB'],loc=(0.015,0.89),handletextpad=0.3,handlelength=0.95,ncol=4,mode='expand',markerscale=50,fontsize=15)

    plt.savefig(fr'{save_path}\\RIN_gain_fb_110mA gain_15pt_font.pdf',bbox_inches='tight')
    plt.savefig(fr'{save_path}\RIN_gain_fb_110mA gain_15pt_font.png',bbox_inches='tight')

In [ ]:
def plot_intensity_data_MHz(directory): #Originally to distinguish between HP detector and Thorlabs 150MHz detector. This one is for HP detector.

    paths = get_paths_intensity(directory)

    no_RIN_values = int(len(paths))

    single_RIN_values = [None for _ in range(no_RIN_values)]
    fb_powers = [None for _ in range(no_RIN_values)]
    polarizations = [None for _ in range(no_RIN_values)]
    

    fb_counter = [0,0,0,0,0,0,0,0]

    no_fb_rel_osc = [0,0,0,0,0,0]
    
    colors = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#e41a1c','#f781bf', '#a65628', '#984ea3',
                  '#999999', '#dede00']
    
    if "Impedance matched" in directory: #Due to wrong impedance matching digitally by ESA an offset is added
            imp_match = np.loadtxt(r"O:\Tech_Photonics\Projects\Narrow Linewidth\MFB Chips\Chip 3 Feedback measurements\FN and RIN measurements\HP RIN DC Block\Background\Average_offset_imp_match.txt",skiprows=2)
            offset_imp = imp_match[0]

    for i, path in enumerate(paths):

        header = get_header(path,length=1)
        # print(header)

        power, fb_level, gain, pol, rbw = esa_header_data(header)

        

        xs, ys = get_data(path) #Hz, dBm

                    
                
        if xs.max() == 1e7 and rbw == 50:
            rbw =100 #The rbw was set by ESA to 100Hz, as 1e5 datapoints is max, so 10MHz/100Hz = 1e5 datapoints


        fb_level = 2*fb_level


        rbw_db = 10*np.log10(rbw) #Hz
        

        fb_dB = ratio_to_db(fb_level,power)

        power /= 40

        ys += -rbw_db #dBm/Hz   Remove raw data dependence on bandwidth 

        voltage_mV = conversion_factor*power + conversion_offset

        if '326.06mV220.0uW' in path:
            voltage_mV = 326.06 #Took no FB another day, as didn't save this properly for impedance matched. The "laser off" background was saved as "no FB".
        print(power,voltage_mV)

        # if power = 0

        ys += -convert_optical_to_electrical2(power,conversion = voltage_mV*1e-3/power)#= 687e-3/220)#1.9/1950)  #Divide with the average optical power (in dB here, so subtract) [dBc/Hz]

        # print("divided with P_0^2", ys, any(np.isnan(ys)))

        if "Impedance matched" in directory:
            ys -= offset_imp #To combat digital conversion by ESA
        
        if ((pol=='none') or (pol=='misaligned') or (pol=='aligned')): 
            plt.figure(0)
            plt.plot(xs,ys, label = f'FB:{fb_dB:.1f}dB, {pol}',alpha = 0.7)
        print(pol,fb_dB)
        if pol == "misaligned" and abs(-7.5-fb_dB)<1:
            plt.figure(1)
            plt.plot(xs,ys, color = '#e41a1c', label = f'FB:{fb_dB:.1f}dB, {pol}',alpha = 0.7)
        elif pol == "aligned" and abs(-7.5-fb_dB)<1:
            plt.figure(1)
            plt.plot(xs,ys, color = '#4daf4a', label = f'FB:{fb_dB:.1f}dB, {pol}',alpha = 1)
        elif pol == 'none' and power>0:
            plt.figure(1)
            plt.plot(xs,ys, color = '#377eb8', label = f'FB:{fb_dB:.1f}dB, {pol}',alpha = 1) # colors = ['#377eb8','#4daf4a','#ff7f00', '#e41a1c']



        diff = xs[1]-xs[0]
        print(xs[10])
        single_RIN_values[i] = linear_to_dB( sum( dB_to_linear(ys[10:]) ) * diff ) #Hz, summing all the datapoints and multiplying with the difference in xs, converting to dB
        fb_powers[i] = fb_dB
        polarizations[i] = pol

    
    for i in range(0,2):#range(3)
        plt.figure(i)
        # plt.title(f'Fb at 110mA: {list_fb[i]} ')
        # plt.legend()
        if i ==0:
            plt.xlabel('Frequency [Hz]')
            plt.xscale('log')
            # plt.ylim([-155,-110])
            plt.grid()
            plt.xlim([1e3,1e7])
        
        else:
            plt.xlabel('Frequency [Hz]')
            plt.xscale('log')
            plt.ylim([-147,-107])
            plt.xlim([1e3,1e7])
            plt.grid()

        plt.ylabel('RIN [dBc/Hz]')
        
        # plt.xlim([0,10])
        # plt.legend()
        plt.xscale('log')

        save_path = r"C:\Users\au617810\OneDrive - Aarhus universitet\Videnskabelig assistent\Measurement spectra and plots\RIN\22-01"

        if i==1:

            plt.legend(['No FB','Aligned -7.5 dB','Misaligned -7.5 dB'],loc=(0.5525,0.405),handletextpad=0.3,handlelength=0.95,fontsize=14)

            plt.savefig(fr'{save_path}\\RIN_gain_fb_MHz_HP.pdf',bbox_inches='tight')
            plt.savefig(fr'{save_path}\\RIN_gain_fb_MHz_HP.png',bbox_inches='tight')

    return fb_powers, single_RIN_values, polarizations

In [ ]:
plt.rcParams.update({'font.size': 15})

plot_intensity_data(r"O:\Tech_Photonics\Projects\Narrow Linewidth\MFB Chips\Chip 3 Feedback measurements\26-09 GHz")

In [ ]:
fb_powers_50imp,single_RIN_values_50imp,polarizations_50imp = plot_intensity_data4(r"O:\Tech_Photonics\Projects\Narrow Linewidth\MFB Chips\Chip 3 Feedback measurements\FN and RIN measurements\HP RIN DC Block\Mismatch impedance")

In [ ]:
fb_powers_imp,single_RIN_values_imp,polarizations_imp = plot_intensity_data4(r"O:\Tech_Photonics\Projects\Narrow Linewidth\MFB Chips\Chip 3 Feedback measurements\FN and RIN measurements\HP RIN DC Block\Impedance matched")